# Installing and importing dependencies

In [1]:
!pip install torch torchvision torchaudio

In [15]:
!pip install selenium

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\DELL\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Encode and Calculate Sentiment

In [8]:
tokens = tokenizer.encode('I love sundays', return_tensors ='pt')

In [9]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.0363, -1.8383, -0.1592,  1.2681,  2.1786]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
int(torch.argmax(result.logits))+1

5

# Collect Comments

In [12]:
r = requests.get('https://www.youtube.com/watch?v=szczpgOEdXs')
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['span'])

In [13]:
results

[<span itemprop="author" itemscope="" itemtype="http://schema.org/Person"><link href="http://www.youtube.com/@NicholasRenotte" itemprop="url"/><link content="Nicholas Renotte" itemprop="name"/></span>,
 <span itemprop="thumbnail" itemscope="" itemtype="http://schema.org/ImageObject"><link href="https://i.ytimg.com/vi/szczpgOEdXs/maxresdefault.jpg" itemprop="url"/><meta content="1280" itemprop="width"/><meta content="720" itemprop="height"/></span>,
 <span id="country-code"></span>]

In [34]:
data=[]

with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
    wait = WebDriverWait(driver,15)
    driver.get("https://www.youtube.com/watch?v=szczpgOEdXs")

    for item in range(10): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(15)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1164\822551651.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
Incompatible release of chromedriver (version 110.0.5481.77) detected in PATH: C:\Users\DELL\AppData\Local\Robomotion\bin\chromedriver.exe


In [35]:
import pandas as pd   
df = pd.DataFrame(data, columns=['comment'])
df.head()

,comment
0,IN\nSkip navigation\nSign in\nBetter Writing\n...
1,
2,I love how you take your time to explain each ...
3,Hey Nik!\nI've been looking left and right for...
4,"Incredible work, love your explanations. Thank..."


In [36]:
def sentiment_score(comment):
    tokens = tokenizer.encode(comment, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [37]:
df['sentiment'] = df['comment'].apply(lambda x:sentiment_score(x[:512]))

In [38]:
df

,comment,sentiment
0,IN\nSkip navigation\nSign in\nBetter Writing\n...,5
1,,4
2,I love how you take your time to explain each ...,5
3,Hey Nik!\nI've been looking left and right for...,5
4,"Incredible work, love your explanations. Thank...",5
5,"Amazing video! Simple, structured, keep up the...",5
6,Awesome video! If its to the extend of your kn...,5
7,amazing video! I mixed the content from your c...,5
8,Love this video! You have an amazing skill in ...,5
9,I love the way you teach. I'm going to do my f...,5


In [39]:
df = df.dropna(axis=1, how='any')
df

,comment,sentiment
0,IN\nSkip navigation\nSign in\nBetter Writing\n...,5
1,,4
2,I love how you take your time to explain each ...,5
3,Hey Nik!\nI've been looking left and right for...,5
4,"Incredible work, love your explanations. Thank...",5
5,"Amazing video! Simple, structured, keep up the...",5
6,Awesome video! If its to the extend of your kn...,5
7,amazing video! I mixed the content from your c...,5
8,Love this video! You have an amazing skill in ...,5
9,I love the way you teach. I'm going to do my f...,5


In [40]:
avg_sentiment_score = df['sentiment'].mean()
avg_sentiment_score

4.4